In [1]:
import pandas as pd
import numpy as np
import pickle

import re 
import datetime

from pybaseball import *

pd.set_option('display.max_columns', 500)

# Background/Introduction

Early in the process of formulating ideas for a project, I decided to address an issue dealing with some form of mechanical failure. Within the oil and gas industry, tubing and pipeline leaks are a costly and sometimes dangerous situation. However, I knew getting the data I needed would be difficult given a two week timeline.

Therefore, I looked to another costly and relevant problem: Injuries in Major League Baseball pitchers.

According to initial data exploration, the amount of money paid to injured pitchers while on the disabled list during the 2018 season is $371,849,373. This is a nontrivial amount of money paid by MLB teams to players that are unable to play.

Additionally, if a team is able to predict that a player is close to potential injury, they would intervene with extra rest or other preventative measures or ultimately trade the pitcher before their status as an uninjured player changes.

More importantly, no player wants to get injured. It's an event usually preceding unfavorable trades, lower value contracts/extensions, painful rehabilitation, and time away from the sport.

It is standard procedure to require players to pass a physical examination with team doctors before a trade is finalized. To summarize, it's an intense and microscopic process that teams and players take very seriously.

Exploring the subject of predicting injuries in the sport, I came across previous attempts:


https://blog.insightdatascience.com/pitcher-prognosis-using-machine-learning-to-predict-baseball-injuries-7f49b36f88e6

https://towardsdatascience.com/predicting-injuries-in-mlb-pitchers-c2e133deca39

It became clear that the problem is inherently difficult to tackle and approaches varied widely. However, I wanted to inject Statcast pitch-level data into the equation. 



# Data Gathering

To approach the problem, I needed to scrape, collect, and eventually combine three data sources: 

    Spottrac : Injury data
    Baseball-Reference: Game level pitching statistics 
    Baseball Savant: Statcast pitch data

A Python script for scraping spotrac data is located in spotrac_scrape.py

Functions for generating baseball-reference game statistics and Statcast data utilize the python package pybaseball:
    https://github.com/jldbc/pybaseball

My scripts are located below.

Additionally, I scraped Brooks Baseball playercard data for eventual NLP work : brooks_scrape.py

In [28]:
playercards = pd.read_pickle('../pickles/2018_cards.pkl')
print(len(playercards))
playercards['justin verlander']

798


'Justin Verlander has thrown 42,514 pitches that have been tracked by the PITCHf/x system between 2007 and 2019, including pitches thrown in the MLB Regular Season and the MLB Postseason. In 2019, he has relied primarily on his Fourseam Fastball (95mph) and Slider (87mph), also mixing in a Curve (79mph) and Change (88mph).BETA Feature:Basic description of 2019 pitches compared to other RHP:His fourseam fastball generates an extremely high number of swings & misses compared to other pitchers\' fourseamers, has good "rising" action, has slight armside run and has slightly above average velo. His slider is an extreme flyball pitch compared to other pitchers\' sliders and is much harder than usual. His curve  has an exceptional bite, is basically never swung at and missed compared to other pitchers\' curves and results in more flyballs compared to other pitchers\' curves. His change (take this with a grain of salt because he\'s only thrown 10 of them in 2019) generates an extremely high nu

In [36]:
def statcast_season(year):
    '''
    function that generates statcast pitch data for a given year(minimum 2015)
    input : 
        year(int) : yyyy
    return: 
        pandas dataframe 
    '''
    # may crash depending on connection and computer
    # run one month at a time if needed
    march = statcast('{}-03-01'.format(year), '{}-03-31'.format(year))
    april = statcast('{}-04-01'.format(year), '{}-04-30'.format(year))
    may = statcast('{}-05-01'.format(year), '{}-05-31'.format(year))
    june = statcast('{}-06-01'.format(year), '{}-06-30'.format(year))
    july = statcast('{}-07-01'.format(year), '{}-07-31'.format(year))
    aug = statcast('{}-08-01'.format(year), '{}-08-31'.format(year))
    sept = statcast('{}-09-01'.format(year), '{}-09-30'.format(year))
    octo = statcast('{}-10-01'.format(year), '{}-10-31'.format(year))
    nov = statcast('{}-11-01'.format(year), '{}-11-30'.format(year))

    total = pd.concat([march,april,may,june,july,aug,sept,octo,nov])
    #drop pitches that weren't classified, not useful for this project
    copy = total.dropna(subset=['pitch_type'])

    return copy

def bbref_season(year):
    '''
    function do generate baseball-reference pitching statistics for each day in given season(year)
    input : 
        year(int) : yyyy
    return: 
        pandas dataframe 
    '''
    start_date = '{}-04-01'.format(year)
    times = list()
    start = datetime.datetime.strptime("{}-03-29".format(year), "%Y-%m-%d")
    end = datetime.datetime.strptime("{}-10-29".format(year), "%Y-%m-%d")
    date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]
    for date in date_generated:
        date = str(date.strftime("%Y-%m-%d"))
        times.append(date)
    frames = pd.DataFrame()
    for time in times:
        try:
            # functionality test
            print(time)
            frame = pitching_stats_range(time, time)
            frame['date'] = time
            frames = frames.append(frame)
        except:
            # except statement for all-star-game errors
            print('error/no mlb games today: ', time)
            continue
    frames['date'] = pd.to_datetime(frames['date'], format="%Y/%m/%d")
    frames.columns = [x.lower().strip() for x in frames.columns]
    frames = frames.set_index(np.arange(len(frames)))
    return frames

In [82]:
games_2018 = bbref_season(2018)

2018-03-29
2018-03-30
2018-03-31
2018-04-01
2018-04-02
2018-04-03
2018-04-04
2018-04-05
2018-04-06
2018-04-07
2018-04-08
2018-04-09
2018-04-10
2018-04-11
2018-04-12
2018-04-13
2018-04-14
2018-04-15
2018-04-16
2018-04-17
2018-04-18
2018-04-19
2018-04-20
2018-04-21
2018-04-22
2018-04-23
2018-04-24
2018-04-25
2018-04-26
2018-04-27
2018-04-28
2018-04-29
2018-04-30
2018-05-01
2018-05-02
2018-05-03
2018-05-04
2018-05-05
2018-05-06
2018-05-07
2018-05-08
2018-05-09
2018-05-10
2018-05-11
2018-05-12
2018-05-13
2018-05-14
2018-05-15
2018-05-16
2018-05-17
2018-05-18
2018-05-19
2018-05-20
2018-05-21
2018-05-22
2018-05-23
2018-05-24
2018-05-25
2018-05-26
2018-05-27
2018-05-28
2018-05-29
2018-05-30
2018-05-31
2018-06-01
2018-06-02
2018-06-03
2018-06-04
2018-06-05
2018-06-06
2018-06-07
2018-06-08
2018-06-09
2018-06-10
2018-06-11
2018-06-12
2018-06-13
2018-06-14
2018-06-15
2018-06-16
2018-06-17
2018-06-18
2018-06-19
2018-06-20
2018-06-21
2018-06-22
2018-06-23
2018-06-24
2018-06-25
2018-06-26
2018-06-27

In [38]:
games_2018.head()

,name,age,#days,lev,tm,g,gs,w,l,sv,ip,h,r,er,bb,so,hr,hbp,era,ab,2b,3b,ibb,gdp,sf,sb,cs,po,bf,pit,str,stl,sts,gb/fb,ld,pu,whip,babip,so9,so/w,date
0,Matt Albers,35,383,MLB-NL,Milwaukee,1,0,NaN,NaN,NaN,1.0,1,0,0,0,0,0,0,0.0,3,0,0,0,1,0,0,0,0,3,14,0.64,0.07,0.00,1.00,0.00,0.00,1.000,0.333,0.0,NaN,2018-03-29
1,Dan Altavilla,25,383,MLB-AL,Seattle,1,0,NaN,NaN,NaN,0.2,0,0,0,0,0,0,0,0.0,1,0,0,0,1,0,0,0,0,1,3,1.00,0.33,0.33,1.00,0.00,0.00,0.000,0.000,0.0,NaN,2018-03-29
2,Jose Alvarez,29,383,MLB-AL,Los Angeles,1,0,NaN,NaN,NaN,0.1,0,0,0,0,1,0,0,0.0,1,0,0,0,0,0,0,0,0,1,4,0.75,0.50,0.00,NaN,NaN,NaN,0.000,NaN,27.0,NaN,2018-03-29
3,Chase Anderson,30,383,MLB-NL,Milwaukee,1,1,NaN,NaN,NaN,6.0,1,0,0,3,6,0,0,0.0,19,0,0,0,0,0,0,0,0,22,97,0.63,0.16,0.11,0.46,0.00,0.15,0.667,0.077,9.0,2.0,2018-03-29
4,Chris Archer,29,383,MLB-AL,Tampa Bay,1,1,NaN,NaN,NaN,6.0,6,4,4,1,6,1,0,6.0,23,3,0,0,0,0,0,0,0,24,81,0.69,0.12,0.15,0.41,0.29,0.18,1.167,0.313,9.0,6.0,2018-03-29


# Data Manipulation

Cleaning the data was an iterative process. Thinking about ways to tie the data together required some foresight and creativity.

Combining the data sources in a meaningful way proved to be a challenge.

Several active pitchers have identical names. Additionally, special characters, nicknames, and and titles vary between data sources. This issue is demonstrated below. I utilized a playerid lookup function in pybaseball to create a dictionary of players - tying regex filtered names with their unique MLB-ID and other industry identifiers. 

With a specific ID for each pitcher, I can join injury, pitching, and statcast data based on a players MLB-ID.

In [84]:
inj_2018 = clean_inj('../data/dl_list_2018.csv', 2018)
inj_2018.head()

,name,posititon,team,injury,days,year,list,startdate,enddate,moneyearned,last,first
0,erichanhold,P,NYM,Oblique,7,2018,60,2018-09-25,10/1,20510,hanhold,eric
1,michaelfulmer,SP,DET,Knee/Oblique,47,2018,60,2018-09-21,10/110-day,145324,fulmer,michael
2,claybuchholz,SP,TOR,Elbow/Oblique,47,2018,60,2018-09-15,10/110-day,379055,buchholz,clay
3,luizgohara,SP,ATL,Shoulder/Ankle,49,2018,60,2018-09-14,10/110-day,146216,gohara,luiz
4,michaelkopech,SP,CHW,Elbow Tommy John,24,2018,60,2018-09-08,10/1,70320,kopech,michael


In [ ]:
def clean_inj(file, year):
    '''
    function to clean injury data created in spotrac_scrape.py
    applies needed regex functions, creates 'start_date' feature for use in applying injury labels
    
    input:
        file: name (csv)
        year: int (yyyy)
    output:
        pandas dataframe
        
    '''
    inj = pd.read_csv(file)
    inj['Year'] = year
    inj['Status'] = inj.Status.str[9:]
    DL_df = inj.copy()
    DL_df['List'], DL_df['Status'] = DL_df['Status'].str.split('|', 1).str
    DL_df['Dates'], DL_df['Status'] = DL_df['Status'].str.split('|', 1).str
    DL_df['StartDate'], DL_df['EndDate'] = DL_df['Dates'].str.split('-', 1).str
    DL_df['Dates'] = DL_df['Dates'].str.replace('..-day', '')
    DL_df['MoneyEarned'] = DL_df['Money_Earned'].str[1:]
    DL_df['MoneyEarned'] = DL_df['MoneyEarned'].str.replace(',', '')
    del DL_df['Dates']
    del DL_df['Status']
    del DL_df['Money_Earned']
    DL_df['Player'] = DL_df['Player'].str.lower()
    DL_df['List'] = DL_df['List'].str.replace('-day', '', regex=True)
    dates = []
    df = DL_df
    df.columns = map(str.lower, df.columns)
    for idx, val in df.iterrows():
        val.startdate = str(val.year)+val.startdate
        dates.append(val.startdate)

    dates = [x.rstrip() for x in dates]
    dates = [x.replace(' ','/') for x in dates]
    df['startdate'] = dates
    df['startdate'] = pd.to_datetime(df['startdate'], infer_datetime_format=True)
    
    lasts = []
    firsts = []

    for row in df.iterrows():
        name = row[1][0]
        name = jr_replace(name)
        name = title_replace(name)
        index = row[0]
        first = name.split()[0]
        first = correction2(first).rstrip()
        firsts.append(first)
        last = name.split()[-1]
        lasts.append(last)
    
    df['last'] = lasts
    df['first'] = firsts
    df['player'] = df['player'].str.lower()
    df['player'] = df['player'].str.replace('.', '')
    df['player'] = df['player'].str.replace("'", '')
    df['player'] = df['player'].str.replace(' ', '')
    df.rename(columns={'player': 'name'}, inplace=True)
    
    return df

def jr_replace(x):
    #removes 'jr' from names
    match = re.sub(r'jr$',"",x)
    return match

def title_replace(x):
    #removes titles
    match = re.sub(r'iii$',"",x)
    return match

def correction2(string):
    #function to make initialed names consistent
    corstr = re.sub('\ +',' ',string)
    final = re.sub('\.','. ',corstr)
    return final


In [60]:
unique_pitchers = pitching_stats_range("2015-03-01", "2018-11-01")

unique_pitchers.columns = map(str.lower, unique_pitchers.columns)

# df = pd.DataFrame()

# for idx, val in unique_pitchers.iterrows():
#     try:
#         first = unique_pitchers['name'][idx].split()[0]
#         last = unique_pitchers['name'][idx].split()[1]
#         data = playerid_lookup(last, first)
#         df = df.append(data)
#     except:
#         print('error: ', first, last)
#         continue

print(len(unique_pitchers))
unique_pitchers.head()

1332


,name,age,#days,lev,tm,g,gs,w,l,sv,ip,h,r,er,bb,so,hr,hbp,era,ab,2b,3b,ibb,gdp,sf,sb,cs,po,bf,pit,str,stl,sts,gb/fb,ld,pu,whip,babip,so9,so/w
1,David Aardsma,33,1332,MLB-NL,Atlanta,33,0,1.0,1.0,NaN,30.2,25,17,16,14,35,6,1,4.70,112,7,0,3,4,1,4,1,0,129,492,0.65,0.14,0.15,0.29,0.35,0.06,1.272,0.264,10.3,2.50
2,Fernando Abad,31,562,MLB-AL,"Boston,Minnesota,Oakland",167,0,5.0,9.0,2.0,138.0,125,61,57,55,123,19,3,3.72,518,22,2,6,12,6,8,4,0,585,2381,0.62,0.18,0.09,0.42,0.26,0.07,1.304,0.277,8.0,2.24
3,A.J. Achter,27,927,MLB-AL,"Los Angeles,Minnesota",38,0,1.0,1.0,NaN,51.0,55,23,23,18,28,11,1,4.06,198,10,0,2,8,1,5,1,0,218,796,0.63,0.11,0.11,0.40,0.26,0.08,1.431,0.275,4.9,1.56
4,Jason Adam,26,242,MLB-AL,Kansas City,31,0,NaN,3.0,NaN,32.1,30,22,22,15,37,9,3,6.12,122,6,1,1,1,2,2,1,0,142,568,0.61,0.12,0.13,0.29,0.22,0.19,1.392,0.269,10.3,2.47
5,Austin Adams,29,931,MLB-AL,Cleveland,47,0,2.0,NaN,1.0,51.2,64,37,34,20,40,7,0,5.92,215,19,4,1,5,0,2,0,0,237,845,0.65,0.14,0.11,0.46,0.27,0.05,1.626,0.339,7.0,2.00


In [76]:
# no results
playerid_lookup('Achter', 'A.J.')

Gathering player lookup table. This may take a moment.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last


In [77]:
# no results
playerid_lookup('Achter', 'A J')

Gathering player lookup table. This may take a moment.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last


In [79]:
# finally...
playerid_lookup('Achter', 'A. J.')

Gathering player lookup table. This may take a moment.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,achter,a. j.,592091,achta001,achteaj01,11387,2014.0,2016.0


In [61]:
def clean_games(season):
    firsts = []
    lasts = []
    season = season
    season.columns = map(str.lower, season.columns)
    season['name'] = season['name'].str.lower()
    
    for row in season.iterrows():
        name = row[1][0]
        name = jr_replace(name)
        name = title_replace(name)
        index = row[0]
        first = name.split()[0]
        first = correction2(first).rstrip()
        firsts.append(first)
        last = name.split()[-1]
        lasts.append(last)
    
    season['first'] = firsts
    season['last'] = lasts

    season['name'] = season['name'].str.replace('.', '')
    season['name'] = season['name'].str.replace("'", '')
    season['name'] = season['name'].str.replace(' ', '')
    season['name'] = season['name'].str.strip()
    season['name'] = season['name'].apply(correction2)
    season['name'] = season['name'].apply(jr_replace)
    season['name'] = season['name'].apply(title_replace)

    return season

In [62]:
pitchers = clean_games(unique_pitchers)
pitchers.head()

,name,age,#days,lev,tm,g,gs,w,l,sv,ip,h,r,er,bb,so,hr,hbp,era,ab,2b,3b,ibb,gdp,sf,sb,cs,po,bf,pit,str,stl,sts,gb/fb,ld,pu,whip,babip,so9,so/w,first,last
1,davidaardsma,33,1332,MLB-NL,Atlanta,33,0,1.0,1.0,NaN,30.2,25,17,16,14,35,6,1,4.70,112,7,0,3,4,1,4,1,0,129,492,0.65,0.14,0.15,0.29,0.35,0.06,1.272,0.264,10.3,2.50,david,aardsma
2,fernandoabad,31,562,MLB-AL,"Boston,Minnesota,Oakland",167,0,5.0,9.0,2.0,138.0,125,61,57,55,123,19,3,3.72,518,22,2,6,12,6,8,4,0,585,2381,0.62,0.18,0.09,0.42,0.26,0.07,1.304,0.277,8.0,2.24,fernando,abad
3,ajachter,27,927,MLB-AL,"Los Angeles,Minnesota",38,0,1.0,1.0,NaN,51.0,55,23,23,18,28,11,1,4.06,198,10,0,2,8,1,5,1,0,218,796,0.63,0.11,0.11,0.40,0.26,0.08,1.431,0.275,4.9,1.56,a. j.,achter
4,jasonadam,26,242,MLB-AL,Kansas City,31,0,NaN,3.0,NaN,32.1,30,22,22,15,37,9,3,6.12,122,6,1,1,1,2,2,1,0,142,568,0.61,0.12,0.13,0.29,0.22,0.19,1.392,0.269,10.3,2.47,jason,adam
5,austinadams,29,931,MLB-AL,Cleveland,47,0,2.0,NaN,1.0,51.2,64,37,34,20,40,7,0,5.92,215,19,4,1,5,0,2,0,0,237,845,0.65,0.14,0.11,0.46,0.27,0.05,1.626,0.339,7.0,2.00,austin,adams


In [67]:
pitchers_ex = pitchers[:10]
pitchers_ex

,name,age,#days,lev,tm,g,gs,w,l,sv,ip,h,r,er,bb,so,hr,hbp,era,ab,2b,3b,ibb,gdp,sf,sb,cs,po,bf,pit,str,stl,sts,gb/fb,ld,pu,whip,babip,so9,so/w,first,last
1,davidaardsma,33,1332,MLB-NL,Atlanta,33,0,1.0,1.0,NaN,30.2,25,17,16,14,35,6,1,4.70,112,7,0,3,4,1,4,1,0,129,492,0.65,0.14,0.15,0.29,0.35,0.06,1.272,0.264,10.3,2.50,david,aardsma
2,fernandoabad,31,562,MLB-AL,"Boston,Minnesota,Oakland",167,0,5.0,9.0,2.0,138.0,125,61,57,55,123,19,3,3.72,518,22,2,6,12,6,8,4,0,585,2381,0.62,0.18,0.09,0.42,0.26,0.07,1.304,0.277,8.0,2.24,fernando,abad
3,ajachter,27,927,MLB-AL,"Los Angeles,Minnesota",38,0,1.0,1.0,NaN,51.0,55,23,23,18,28,11,1,4.06,198,10,0,2,8,1,5,1,0,218,796,0.63,0.11,0.11,0.40,0.26,0.08,1.431,0.275,4.9,1.56,a. j.,achter
4,jasonadam,26,242,MLB-AL,Kansas City,31,0,NaN,3.0,NaN,32.1,30,22,22,15,37,9,3,6.12,122,6,1,1,1,2,2,1,0,142,568,0.61,0.12,0.13,0.29,0.22,0.19,1.392,0.269,10.3,2.47,jason,adam
5,austinadams,29,931,MLB-AL,Cleveland,47,0,2.0,NaN,1.0,51.2,64,37,34,20,40,7,0,5.92,215,19,4,1,5,0,2,0,0,237,845,0.65,0.14,0.11,0.46,0.27,0.05,1.626,0.339,7.0,2.00,austin,adams
6,austinadams,27,353,MLB-NL,Washington,8,0,NaN,NaN,NaN,6.0,5,4,2,11,10,0,1,3.00,23,1,0,0,0,1,0,0,0,36,155,0.54,0.21,0.11,0.43,0.29,0.07,2.667,0.357,15.0,0.91,austin,adams
7,chanceadams,23,198,MLB-AL,New York,3,1,NaN,1.0,NaN,7.2,8,7,6,4,4,3,0,7.04,30,1,0,0,2,0,0,0,0,34,153,0.60,0.14,0.05,0.38,0.31,0.00,1.565,0.217,4.7,1.00,chance,adams
8,nathanadcock,27,1360,MLB-NL,Cincinnati,13,0,1.0,2.0,NaN,18.0,15,12,12,12,13,3,1,6.00,68,5,1,0,0,1,5,1,0,83,338,0.54,0.15,0.08,0.50,0.30,0.04,1.500,0.226,6.5,1.08,nathan,adcock
9,timadleman,29,574,MLB-NL,Cincinnati,43,33,9.0,15.0,NaN,192.0,188,111,106,71,155,42,11,4.97,724,45,4,2,16,5,10,2,3,818,3189,0.63,0.16,0.10,0.36,0.26,0.08,1.349,0.274,7.3,2.18,tim,adleman
10,jeremyaffeldt,36,1290,MLB-NL,San Francisco,52,0,2.0,2.0,NaN,35.1,43,24,23,14,21,6,2,5.86,147,6,0,2,4,0,4,0,0,163,574,0.62,0.19,0.07,0.57,0.25,0.05,1.613,0.308,5.3,1.50,jeremy,affeldt


In [80]:
#example of building name-mlbid dataframe on subset of data

ids = pd.DataFrame()

for idx, val in pitchers_ex.iterrows():
    first = str(pitchers['first'][idx])
    last = str(pitchers['last'][idx])
    key = playerid_lookup(last, first)
    print(first, last)
    ids = ids.append(key)

Gathering player lookup table. This may take a moment.
david aardsma
Gathering player lookup table. This may take a moment.
fernando abad
Gathering player lookup table. This may take a moment.
a. j. achter
Gathering player lookup table. This may take a moment.
jason adam
Gathering player lookup table. This may take a moment.
austin adams
Gathering player lookup table. This may take a moment.
austin adams
Gathering player lookup table. This may take a moment.
chance adams
Gathering player lookup table. This may take a moment.
nathan adcock
Gathering player lookup table. This may take a moment.
tim adleman
Gathering player lookup table. This may take a moment.
jeremy affeldt


In [74]:
ids

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,aardsma,david,430911,aardd001,aardsda01,1902,2004.0,2015.0
0,abad,fernando,472551,abadf001,abadfe01,4994,2010.0,2017.0
0,achter,a. j.,592091,achta001,achteaj01,11387,2014.0,2016.0
0,adam,jason,592094,adamj002,adamja01,11861,2018.0,2018.0
0,adams,austin,613534,adama002,adamsau02,13801,2017.0,2018.0
1,adams,austin,-1,NaN,NaN,-1,NaN,NaN
2,adams,austin,-1,NaN,NaN,-1,NaN,NaN
3,adams,austin,542866,adama001,adamsau01,7411,2014.0,2016.0
4,adams,austin,-1,NaN,NaN,-1,NaN,NaN
0,adams,austin,613534,adama002,adamsau02,13801,2017.0,2018.0


It turns out adding MLB-IDs to filtered 'names' wasn't going to be as straightforward as I expected.